# Taravel_tide Data hochladen für Mastery projekt

In [1]:
# Notebook: notebooks/load_dat.ipynb
import os
import sys
import pandas as pd  # noqa: F401

# ============================================================
# ✅ Add the project root to sys.path (not /core)
# ============================================================
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)



Project root added to sys.path: /Users/claudiatagbo/Masterschool/traveltide


In [2]:
# Verify that core is importable
try:
    import core
    print(":weißes_häkchen: core module found at:", core.__file__)
except ModuleNotFoundError:
    print(":x: core module not found. Check your sys.path!")

:weißes_häkchen: core module found at: None


In [3]:
#from core import load_table
import core



In [4]:
from core.load_data import load_table

✅ Connected to PostgreSQL database.


## ** 1. Verständnis der Datenstruktur

In [10]:
users_df = load_table(data_type='raw', table_name='users')
session_df = load_table(data_type='raw', table_name='sessions')
flights_df = load_table(data_type='raw', table_name='flights')
hotel_df = load_table(data_type='raw', table_name='hotels')


 Lade Tabelle 'users' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/raw/users.csv
 CSV geladen. Zeilen: 1020926
 Lade Tabelle 'sessions' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/raw/sessions.csv
 CSV geladen. Zeilen: 5408063
 Lade Tabelle 'flights' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/raw/flights.csv
 CSV geladen. Zeilen: 1901038
 Lade Tabelle 'hotels' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/raw/hotels.csv
 CSV geladen. Zeilen: 1918617


# Datenmodellbeschreibung für TravelTide
Die Datenbank von TravelTide umfasst insgesamt 1.020.926 Nutzerkonten, die gemeinsam 5.408.063 Sitzungen generiert haben. Aus diesen Interaktionen entstanden 1.901.038 Flug- und 1.918.617 Hotelbuchungen.
Im Zentrum des Modells steht die sessions-Tabelle – sie fungiert als primäre Faktentabelle und erfasst alle transaktionsbezogenen Aktivitäten, die während einzelner Nutzersitzungen auftreten. Die users-Tabelle ergänzt dieses Modell als Dimensionstabelle, indem sie stabile demografische Merkmale bereitstellt und damit eine tiefere Analyse des Nutzerverhaltens ermöglicht.
Zusätzlich liefern die Tabellen flights und hotels detaillierte Informationen zu einzelnen Buchungen. Sie sind eng mit den Sitzungsdaten verknüpft und erlauben präzise Einblicke in Reisegewohnheiten, Preisstrukturen und Buchungstrends.

## **2. Erste Prüfung der Datenqualität
da die Daten zu große ist , wird die Datenqualität in SQL geprüft werden und hier weiter analysiert

In [6]:
from core.eda import missing_values_summary

In [7]:
missing_values_summary(users_df)

,column,missing_count,missing_percent
user_id,user_id,0,0.0
birthdate,birthdate,0,0.0
gender,gender,0,0.0
married,married,0,0.0
has_children,has_children,0,0.0
home_country,home_country,0,0.0
home_city,home_city,0,0.0
home_airport,home_airport,0,0.0
home_airport_lat,home_airport_lat,0,0.0
home_airport_lon,home_airport_lon,0,0.0


In [11]:
missing_values_summary(session_df)

,column,missing_count,missing_percent
session_id,session_id,0,0.00
user_id,user_id,0,0.00
trip_id,trip_id,3072218,56.81
session_start,session_start,0,0.00
session_end,session_end,0,0.00
flight_discount,flight_discount,0,0.00
hotel_discount,hotel_discount,0,0.00
flight_discount_amount,flight_discount_amount,4522267,83.62
hotel_discount_amount,hotel_discount_amount,4716683,87.22
flight_booked,flight_booked,0,0.00


In [12]:
missing_values_summary(flights_df)

,column,missing_count,missing_percent
trip_id,trip_id,0,0.00
origin_airport,origin_airport,0,0.00
destination,destination,0,0.00
destination_airport,destination_airport,0,0.00
seats,seats,0,0.00
return_flight_booked,return_flight_booked,0,0.00
departure_time,departure_time,0,0.00
return_time,return_time,88734,4.67
checked_bags,checked_bags,0,0.00
trip_airline,trip_airline,0,0.00


In [13]:
missing_values_summary(hotel_df)

,column,missing_count,missing_percent
trip_id,trip_id,0,0.0
hotel_name,hotel_name,0,0.0
nights,nights,0,0.0
rooms,rooms,0,0.0
check_in_time,check_in_time,0,0.0
check_out_time,check_out_time,0,0.0
hotel_per_room_usd,hotel_per_room_usd,0,0.0


**TravelTide Datenqualität Analyse**

Die Datenprüfung zeigte, dass die **User-Tabelle mit 49.211 Einträgen** vollständig und fehlerfrei ist – ein starkes Fundament für alle Analysen. In den **4,9 Millionen Session-Datensätzen** fehlen jedoch **über 3 Millionen trip_id-Werte (56,8 %)** sowie **mehr als 4,7 Millionen Rabattbeträge (bis zu 87 %)**, was auf unvollständige Buchungsvorgänge hinweist. Dagegen sind die **Flight- und Hotel-Daten fast lückenlos**, mit nur **88.734 fehlenden Rückflugdaten (4,7 %)**, die wahrscheinlich One-Way-Reisen widerspiegeln.


In [8]:
sample_users = users_df.sample(15000, random_state=42)

In [9]:
# load Base_session from sql code
base_session = load_table(data_type='sql', table_name='session_base')

 Lade Tabelle 'session_base' aus SQL-Datei: /Users/claudiatagbo/Masterschool/traveltide/core/../sql/session_base.sql
📄 Executing SQL from file: /Users/claudiatagbo/Masterschool/traveltide/core/../sql/session_base.sql
✅ Query executed. 49211 rows retrieved.
 SQL-Abfrage erfolgreich. Zeilen: 49211
 /Users/claudiatagbo/Masterschool/traveltide/core/../data/processed/session_base.csv


In [ ]:
# Erweiterter Sitzungsdatensatz gemäß Elenas Kohortenlogik
base_sessions = load_table(data_type='processed', table_name='session_base')

 Lade Tabelle 'session_base' aus CSV: /Users/claudiatagbo/Masterschool/traveltide/core/../data/processed/session_base.csv
 CSV geladen. Zeilen: 49211


Der **erweiterte Sitzungsdatensatz (`base_sessions`)** bündelt die Aktivitäten jener **aktiven Nutzer*innen**, die im Jahr 2023 **mehr als sieben Sitzungen** absolviert haben. Durch die SQL-Abfrage wurden die ursprünglichen **5,4 Millionen Sitzungen** gezielt auf diese Nutzer*innen gefiltert und mit den entsprechenden **Flug-, Hotel- und demografischen Informationen** verknüpft. Das resultierende Dataset umfasst **49.211 Einträge** und wurde erfolgreich unter `/data/processed/session_base.csv` gespeichert. Diese Verdichtung **reduziert das Datenvolumen deutlich**, bleibt aber **repräsentativ für die User-Journey** der aktivsten Reisenden und ist dadurch **effizient analysierbar**.


In [15]:
missing_values_summary(base_sessions)

,column,missing_count,missing_percent
session_id,session_id,0,0.00
user_id,user_id,0,0.00
trip_id,trip_id,32509,66.06
session_start,session_start,0,0.00
session_end,session_end,0,0.00
page_clicks,page_clicks,0,0.00
flight_discount,flight_discount,0,0.00
flight_discount_amount,flight_discount_amount,40929,83.17
hotel_discount,hotel_discount,0,0.00
hotel_discount_amount,hotel_discount_amount,43006,87.39



Die Analyse des **erweiterten Sitzungsdatensatzes (`base_sessions`)** zeigt, dass **demografische Daten** und **Sitzungsinformationen** vollständig vorliegen, während bei den **Buchungsdetails** deutliche Lücken bestehen. Konkret fehlen **66 % der `trip_id`-Werte**, rund **71 % der Fluginformationen** (`origin_airport`, `destination`, `seats`, `departure_time`) sowie etwa **70 % der Hoteldaten** (`hotel_name`, `nights`, `rooms`). Diese Ergebnisse weisen darauf hin, dass der Datensatz sich gut für die **Analyse des Nutzerverhaltens aktiver Reisender** eignet, jedoch **Buchungsinformationen nur eingeschränkt verfügbar** sind und daher bei **tiefergehenden Reiseanalysen mit Vorsicht interpretiert** werden sollten.


In [23]:
def show_summary_stats(df: pd.DataFrame) -> None:
        """
        Displays basic summary statistics.
        """
        print(" Summary Statistics:")
        display(df.describe().transpose().round(2))

In [24]:
show_summary_stats(base_session)

 Summary Statistics:


,count,mean,min,25%,50%,75%,max,std
user_id,49211.0,545282.694946,23557.0,517119.0,540308.0,573922.0,844489.0,64640.047648
session_start,49211,2023-03-21 11:25:24.870191616,2023-01-04 00:01:00,2023-02-05 22:31:30,2023-03-09 11:04:00,2023-04-28 11:23:00,2023-07-28 19:58:52,NaN
session_end,49211,2023-03-21 11:28:32.122520576,2023-01-04 00:04:23,2023-02-05 22:34:10.500000,2023-03-09 11:06:35,2023-04-28 11:25:11.500000,2023-07-28 20:08:52,NaN
page_clicks,49211.0,17.588791,1.0,6.0,13.0,22.0,566.0,21.495987
flight_discount_amount,8282.0,0.139864,0.05,0.1,0.1,0.2,0.6,0.083914
hotel_discount_amount,6205.0,0.112192,0.05,0.05,0.1,0.15,0.45,0.062119
home_airport_lat,49211.0,38.427222,21.316,33.818,39.103,42.238,61.251,6.185962
home_airport_lon,49211.0,-94.183762,-157.927,-112.383,-90.035,-79.37,-63.499,18.065868
seats,14270.0,1.215207,1.0,1.0,1.0,1.0,8.0,0.585891
departure_time,14270,2023-04-15 17:07:40.154169600,2023-01-07 07:00:00,2023-02-15 15:00:00,2023-03-23 16:00:00,2023-05-23 14:45:00,2024-07-16 07:00:00,NaN
